In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

C:\Users\prit4\AppData\Local\Temp\ipykernel_11384\3674435951.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dataset_path = "dataset.csv"
df = pd.read_csv(dataset_path)

# X = df[['month', 'day', 'time','humidity','tempC']]
# y = df['fan_speed']


In [3]:
df['month'] /= 12
df['day'] /= 31
df['time'] /= 24
df['humidity'] /= 100
df['tempC'] /= 50

# Check the updated DataFrame
print(df.head())

      month       day      time  humidity  tempC  fan_speed
0  0.083333  0.032258  0.000000      0.74   0.36          0
1  0.083333  0.032258  0.041667      0.78   0.36          0
2  0.083333  0.032258  0.083333      0.82   0.34          0
3  0.083333  0.032258  0.125000      0.85   0.34          0
4  0.083333  0.032258  0.166667      0.83   0.34          0


In [4]:
scaled_df = pd.DataFrame()
scaled_df['month'] = df['month'] / 12
scaled_df['day'] = df['day'] / 31
scaled_df['time'] = df['time'] / 24
scaled_df['humidity'] = df['humidity'] / 100
scaled_df['tempC'] = df['tempC'] / 50
print(scaled_df.head())

      month       day      time  humidity   tempC
0  0.006944  0.001041  0.000000    0.0074  0.0072
1  0.006944  0.001041  0.001736    0.0078  0.0072
2  0.006944  0.001041  0.003472    0.0082  0.0068
3  0.006944  0.001041  0.005208    0.0085  0.0068
4  0.006944  0.001041  0.006944    0.0083  0.0068


In [5]:
x = scaled_df[['month', 'day', 'time','humidity','tempC']]
y = df['fan_speed']

In [6]:


# Convert features and target variable to PyTorch tensors
x_tensor = torch.FloatTensor(x.values)
y_tensor = torch.LongTensor(y.values)  # Assuming 'fan_speed' is categorical

# Split data into training and testing sets
test_size = 0.2
random_state = 0
x_train_tensor, x_test_tensor, y_train_tensor, y_test_tensor = train_test_split(
    x_tensor, y_tensor, test_size=test_size, random_state=random_state
)

# Create a training dataset
training_data = TensorDataset(x_train_tensor, y_train_tensor)
validation_data=TensorDataset(x_test_tensor, y_test_tensor)
# Define batch size
batch_size = 32  # Adjust as needed

# Create a DataLoader for training data
training_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
validation_loader  = DataLoader(validation_data, batch_size=batch_size, shuffle=True)



In [7]:
class speed_predictor(nn.Module):
    def __init__(self, input_features=5, hidden1=50, hidden2=50, output_features=1):
        super(speed_predictor, self).__init__()
        self.fc1 = nn.Linear(input_features, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, output_features)
        
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
def validate(model, validation_loader, loss_function):
    model.eval()  # Set model to evaluation mode
    total_loss = 0.0
    with torch.no_grad():  # Disable gradient calculation during validation
        for inputs, labels in validation_loader:
            outputs = model(inputs)
            labels = labels.unsqueeze(1)
            loss = loss_function(outputs.float(), labels.float())
            total_loss += loss.item()

    average_loss = total_loss / len(validation_loader)
    return average_loss

In [9]:
model = speed_predictor()

In [13]:
number_of_epochs = 5
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [14]:
for epoch in range(number_of_epochs):
    running_loss = 0.0
    for i, data in enumerate(training_loader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        labels = labels.unsqueeze(1)
        loss = loss_function(outputs.float(), labels.float())
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if i % 2000 == 99:  # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    # Validation at the end of each epoch
    # validation_loss = validate(model, validation_loader, loss_function)
    # print(f'Validation Loss after epoch {epoch + 1}: {validation_loss}')

print('Finished Training')

[1,   100] loss: 0.190
[2,   100] loss: 0.154
[3,   100] loss: 0.167
[4,   100] loss: 0.154
[5,   100] loss: 0.163
Finished Training


In [28]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in validation_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

model_accuracy = correct / total
print(model_accuracy)

0.06488332384746727
